In [6]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from transformers import BertTokenizer, Trainer, BertForSequenceClassification, TrainingArguments
from datasets import Dataset
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [29]:
data = pd.read_csv("all_ECB_speeches.csv", sep="|", encoding="utf-8")
data.head()

,date,speakers,title,subtitle,contents
0,2024-02-26,Christine Lagarde,European Parliament plenary debate on the ECB ...,"Speech by Christine Lagarde, President of the ...",SPEECH European Parliament plenary debate ...
1,2024-02-23,Isabel Schnabel,Has the fight against inflation been won?,"Slides by Isabel Schnabel, Member of the Execu...",NaN
2,2024-02-16,Isabel Schnabel,From laggard to leader? Closing the euro area’...,Inaugural lecture of the EMU Lab by Isabel Sch...,SPEECH From laggard to leader? Closing the ...
3,2024-02-15,Philip R. Lane,The banking channel of monetary policy,"Slides by Philip R. Lane, Member of the Execut...",NaN
4,2024-02-15,Christine Lagarde,Hearing of the Committee on Economic and Monet...,"Speech by Christine Lagarde, President of the ...",SPEECH Hearing of the Committee on Economic...


In [5]:
model = BertForSequenceClassification.from_pretrained("ProsusAI/finbert", num_labels=3)
tokenizer = BertTokenizer.from_pretrained("ProsusAI/finbert")

In [32]:
data = data.dropna(subset=data.columns)
data.size
#data.head()

13415

In [33]:
print("Número de linhas não nulas: ",data["contents"].count())
data["contents"].head()

Número de linhas não nulas:  2683


0      SPEECH   European Parliament plenary debate ...
2      SPEECH  From laggard to leader? Closing the ...
4      SPEECH  Hearing of the Committee on Economic...
6      SPEECH       Preserving people’s freedom to ...
7      SPEECH  Monetary policy, financial stability...
Name: contents, dtype: object

In [40]:
for content in data["contents"]:
    content = tokenizer(content)
data["contents"].head()

0      SPEECH   European Parliament plenary debate ...
2      SPEECH  From laggard to leader? Closing the ...
4      SPEECH  Hearing of the Committee on Economic...
6      SPEECH       Preserving people’s freedom to ...
7      SPEECH  Monetary policy, financial stability...
Name: contents, dtype: object

In [39]:
predictions = []
for content in data["contents"]:
    predictions.append(model(**content))

TypeError: BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,),